# Limpieza y Dataset Model Ready

Objetivo: crear el unico dataset permitido para entrenamiento, con trazabilidad completa desde el dataset original.
Regla: no entrenar modelos en esta fase.


## 1. Cargar dataset auditado


In [1]:
import pandas as pd
from pathlib import Path

dataset_path = Path('..') / 'DATASET ORIGINAL' / 'DATASET_FINAL_HACKATHON_2026.parquet'
df = pd.read_parquet(dataset_path)
df.shape


(35669175, 25)

## 2. Eliminar variables con fuga de informacion
Se eliminan columnas que contienen informacion posterior al despegue (ej. DEP_DELAY).


In [2]:
leakage_cols = ['DEP_DELAY']
df = df.drop(columns=[c for c in leakage_cols if c in df.columns])
df.shape


(35669175, 24)

## 3. Resolver redundancias
Se eliminan columnas derivadas o duplicadas para evitar multicolinealidad.


In [3]:
redundant_cols = [
    'ORIGIN_CITY_NAME',
    'DEST_CITY_NAME',
    'YEAR',
    'MONTH',
    'DAY_OF_MONTH',
    'DEP_HOUR',
    'CRS_DEP_TIME',
    'distance_bin',
    'DIST_MET_KM',
]
df = df.drop(columns=[c for c in redundant_cols if c in df.columns])
df.shape


(35669175, 15)

## 4. Corregir valores fuera de dominio


In [4]:
# Corregir valores de minuto de dia (1440 -> 0)
if 'sched_minute_of_day' in df.columns:
    df.loc[df['sched_minute_of_day'] == 1440, 'sched_minute_of_day'] = 0

# Precipitacion negativa no es valida
if 'PRECIP_1H' in df.columns:
    df.loc[df['PRECIP_1H'] < 0, 'PRECIP_1H'] = 0


## 5. Tratar outliers (justificado)
Se aplica winsorizacion al 1% y 99% en variables numericas sensibles para robustez del MVP.


In [5]:
winsor_cols = ['TEMP', 'WIND_SPD', 'PRECIP_1H', 'DISTANCE']
for col in winsor_cols:
    if col in df.columns:
        low, high = df[col].quantile([0.01, 0.99])
        df[col] = df[col].clip(lower=low, upper=high)


## 6. Guardar dataset final


In [6]:
output_path = Path('outputs') / 'flights_model_ready.parquet'
df.to_parquet(output_path, index=False)
output_path


WindowsPath('outputs/flights_model_ready.parquet')